In [1]:
#@title
from IPython.display import HTML, display

title = "<h1 style='font-family:Comic Sans MS; font-size:50px; color:pink; text-shadow: 2px 2px #aaaaaa; text-align:center'>Natural Language Processing: Rule-based approach</h1>"
display(HTML(title))


## 🌸 Libraries 

In [2]:
import sys
!{sys.executable} -m pip install spacy
!{sys.executable} -m pip install negspacy
!{sys.executable} -m pip install spacy_stanza 

Defaulting to user installation because normal site-packages is not writeable
     |████████████▊                   | 2.6 MB 3.1 MB/s eta 0:00:02^C

ERROR: Operation cancelled by user
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
  distutils: /private/var/folders/j8/64lnvrmj50q5dv_5dj_ztbz40000gn/T/pip-build-env-0szewrf_/normal/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  distutils: /private/var/folders/j8/64lnvrmj50q5dv_5dj_ztbz40000gn/T/pip-build-env-0szewrf_/normal/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  user = False
  home = None
  root = None
  prefix = '/private/var/folders/j8/64lnvrmj50q5dv_5dj_ztbz40000gn/T/pip-build-env-0szewrf_/normal'
  distutils: /private/var/folders/j8/64lnvrmj50q5dv_5dj_ztbz40000gn/T/pip-build-env-0szewrf_/overlay/lib/python3.9/

In [ ]:
import json
import spacy # to build a nlp pipeline
import stanza # for named entity recognition
import spacy_stanza
from negspacy.negation import Negex
from negspacy.termsets import termset # to customize negation terms
import pandas as pd
from spacy.language import Language
import numpy as np
from prettytable import PrettyTable

## 🌸  Loading data 

In [ ]:
#Import json file, load data

with open("negacio_uab_revised_version.json", "r", encoding="utf-8") as f:
    data = json.load(f)

## 🌸 Loading the spacy_stanza pipeline 

In [ ]:
nlp = spacy_stanza.load_pipeline('es')

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: es (Spanish):
| Processor    | Package  |
---------------------------
| tokenize     | ancora   |
| mwt          | ancora   |
| pos          | ancora   |
| lemma        | ancora   |
| constituency | combined |
| depparse     | ancora   |
| sentiment    | tass2020 |
| ner          | conll02  |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: constituency
INFO:stanza:Loading: depparse
INFO:stanza:Loading: sentiment
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


## 🌸 Functions definitions 

Function to cut the text, useful if we do not want certian characters. Not strictly necessary.

In [ ]:
def cut_text(text):
 
  # Find the index of the word to start the cut from
  word_to_cut_from = "informe"
  start_index = text.find(word_to_cut_from)

  # Use string slicing to cut the string from the start index to the end
  text = text[start_index:]

  return text


Function to detect cues of both Negation (NEG) and Uncertainty (UNC).

In [ ]:
def detect_cues(text):

    doc = nlp(text)
    negation_indices = set() #Empty set, negation ind will be stored here
    uncertainty_indices = set()

    for token in doc:

        if token.lemma_ in ["no", "sin", "tampoco", "afebril", "ex", "negativo", "ausencia", "descarta",
                            "niega", "inespecificos", "tampoco", "asintomatico", "imposibilidad","atipicos", "sense",
                            "neg","ausente","retira","incapacidad", "falta", "insuficiencia", "inestabilidad",
                            "negativas","desaparicion","desconocen","rechaza","negativa"]:

          negation_indices.add(token.i) 
        
        if token.lemma_ in ["sospecha","duda", "probable", "compatible", "podria", "aparentemente", 
                            "sugiriendo", "dudosa", "sugestivo"]:
                            
          uncertainty_indices.add(token.i) 

    labeled_tokens = []

    for i, token in enumerate(doc):
        if i in negation_indices:

            labeled_tokens.append((token, "NEG"))

        elif i in uncertainty_indices:
            labeled_tokens.append((token, "UNC"))

        else:
            labeled_tokens.append((token, "O")) #Other if it is not

    return labeled_tokens, negation_indices, uncertainty_indices

Function to detect scopes.

In [ ]:
def detect_scopes(text):

    doc = nlp(text)
    labels, negation_indices, uncertainty_indices = detect_cues(text)
    scopes = [] #Store the cue-scope pairs
      
    for token in doc:
        if token.i in negation_indices or token.i in uncertainty_indices: #Token is a cue?

            # Find the nearest verb or adjective that the cue modifies, using dependency parse tree
            target_token = token.head
            while target_token.pos_ not in ["VERB", "ADJ"] and target_token != target_token.head:
                target_token = target_token.head
            
            # Scope  -> Subtree of doc, wehere head = Target token
            scope_start = target_token.left_edge.i
            scope_end = target_token.right_edge.i + 1
            scope = doc[scope_start:scope_end]
        
            if token.i in negation_indices:
              for index in range(scope_start, scope_end):
                if labels[index][1] != "NEG":
                  labels[index] = (labels[index][0], "NSCO")

            else:
              for index in range(scope_start, scope_end):
                if labels[index][1] != "UNC":
                  labels[index] = (labels[index][0], "USCO")


            scopes.append((token.lower_, scope.text))

    
    return scopes, labels

Function to compute the labels using annotations from json file (true_labels).

PARAMETERS: 

1. data - > All data extracted from json file.
2. num_text - > Number of desired text.

In [ ]:
def test_data(data, num_text):

  text = data[num_text]["data"]["text"]
  labels = list(np.full(len(text), "O", dtype=str))

  for value in data[num_text]["predictions"][0]["result"]:
      
      if value["value"]["labels"][0]== "NEG":

        start = value["value"]["start"]
        end = value["value"]["end"]

        #print(text[start:end])
        labels[start:end] = ["NEG"] * (end - start)
        #print(labels[start:end-1])
      
      elif value["value"]["labels"][0]== "UNC":

        start = value["value"]["start"]
        end = value["value"]["end"]

        #print(text[start:end])
        labels[start:end] = ["UNC"] * (end - start)
        #print(labels[start:end-1])

  print("Completed (0)")

  doc = nlp(text)

  print("Completed (1)")
  final_labels = []
  
  # Use a counter variable to keep track of the current position in the labels list
  i = 0

  for token in doc:
    while text[i] == " ": #IMPORTANT to handle spaces
      i+=1

    label = labels[i] # Get the label for the current token

    #print(token, label, i)

    final_labels.append((token.text, label))
    i += len(token)

  return final_labels

Same function as before, but now taking the scopes into account.

In [ ]:
def test_data_with_scopes(data, num_text):

  text = data[num_text]["data"]["text"]
  labels = list(np.full(len(text), "O", dtype=str))

  for value in data[num_text]["predictions"][0]["result"]:
      
      if value["value"]["labels"][0]== "NEG":

        start = value["value"]["start"]
        end = value["value"]["end"]

        #print(text[start:end])
        labels[start:end] = ["NEG"] * (end - start)
        #print(labels[start:end-1])
      
      elif value["value"]["labels"][0]== "UNC":

        start = value["value"]["start"]
        end = value["value"]["end"]

        #print(text[start:end])
        labels[start:end] = ["UNC"] * (end - start)
        #print(labels[start:end-1])

      elif value["value"]["labels"][0]== "USCO":

        start = value["value"]["start"]
        end = value["value"]["end"]

        #print(text[start:end])
        labels[start:end] = ["USCO"] * (end - start)
        #print(labels[start:end-1])
      
      elif value["value"]["labels"][0]== "NSCO":

        start = value["value"]["start"]
        end = value["value"]["end"]

        #print(text[start:end])
        labels[start:end] = ["NSCO"] * (end - start)
        #print(labels[start:end-1])

  print("Completed (0)")

  doc = nlp(text)

  print("Completed (1)")
  final_labels = []
  
  # Use a counter variable to keep track of the current position in the labels list
  i = 0

  for token in doc:
    while text[i] == " ": #IMPORTANT to handle spaces
      i+=1

    label = labels[i] # Get the label for the current token

    #print(token, label, i)

    final_labels.append((token.text, label))
    i += len(token)

  return final_labels

The following function is for evaluating the model in negation and uncertainty cues detection. 

PARAMETERS:

1. prdicted_labels - > Labels predicted after processing
2. true_labels - > Labels extracted from annotations
3. cuetype - > "NEG" or "UNC" 

In [ ]:
def evaluating_model (predicted_labels, true_labels, cuetype):

    true_negatives_neg = 0
    true_positives_neg = 0
    false_positives_neg = 0
    false_negatives_neg = 0
    
    for true_value, pred_value in zip(true_labels, predicted_labels):

      #print(true_value[1], pred_value[1])

      if true_value[1] == cuetype and pred_value[1] ==cuetype:
        true_positives_neg +=1
            
      elif true_value[1] =="O" and pred_value[1] =="O":
        true_negatives_neg +=1

      elif true_value[1]=="O" and pred_value[1]==cuetype:
        false_positives_neg+=1
      
      else:
        false_negatives_neg+=1
      
    print("True Negatives:",true_negatives_neg)
    print("True Positives:", true_positives_neg)
    print("False Positives:", false_positives_neg)
    print("False Negatives:",false_negatives_neg)

    # Measures

    denominator = true_positives_neg + false_positives_neg

    if denominator == 0:
        precision = 0
    else:
        precision = true_positives_neg / denominator

    denominator = true_positives_neg + false_negatives_neg

    if denominator == 0:
        recall = 0
    else:
        recall = true_positives_neg / denominator

    denominator = true_positives_neg + true_negatives_neg + false_positives_neg + false_negatives_neg

    if denominator == 0:
        f1_score = 0
        accuracy = 0
    else:
        f1_score = 2 * (precision * recall) / (precision + recall)
        accuracy = (true_positives_neg + true_negatives_neg) / denominator
    
    print("Evalutaion for", cuetype, ":")

    x = PrettyTable()
    x.field_names = ["Measure", "Score"]
    x.add_row(["Precision", f"{precision:.2f}"])
    x.add_row(["Recall", f"{recall:.2f}"])
    x.add_row(["F1-score", f"{f1_score:.2f}"])
    x.add_row(["Accuracy", f"{accuracy:.2f}"])

    print(x)



Same function as before, but now taking the scopes into account.

In [ ]:
def final_evaluation (predicted_labels, true_labels):

    true_negatives = 0
    true_positives= 0
    false_positives = 0
    false_negatives = 0
    
    for true_value, pred_value in zip(true_labels, predicted_labels):

      #print(true_value[1], pred_value[1])

      if true_value[1] == pred_value[1] and true_value[1]!= "O":
        true_positives+=1
            
      elif true_value[1] == pred_value[1]:
        true_negatives=1

      elif true_value[1]=="O" and pred_value[1]!="O":
        false_positives+=1
      
      else:
        false_negatives+=1
      
    print("True Negatives:",true_negatives)
    print("True Positives:", true_positives)
    print("False Positives:", false_positives)
    print("False Negatives:",false_negatives)

    # Measures

    denominator = true_positives + false_positives

    if denominator == 0:
        precision = 0
    else:
        precision = true_positives / denominator

    denominator = true_positives + false_negatives

    if denominator == 0:
        recall = 0
    else:
        recall = true_positives / denominator

    denominator = true_positives + true_negatives + false_positives + false_negatives

    if denominator == 0:
        f1_score = 0
        accuracy = 0
    else:
        f1_score = 2 * (precision * recall) / (precision + recall)
        accuracy = (true_positives + true_negatives) / denominator

    x = PrettyTable()
    x.field_names = ["Measure", "Score"]
    x.add_row(["Precision", f"{precision:.2f}"])
    x.add_row(["Recall", f"{recall:.2f}"])
    x.add_row(["F1-score", f"{f1_score:.2f}"])
    x.add_row(["Accuracy", f"{accuracy:.2f}"])

    print(x)

In [ ]:
def report(data):

  i = 0
  true_negatives = 0
  true_positives = 0
  false_positives = 0
  false_negatives = 0

  for d in data:

    print("Playing with text", i)

    text = d["data"]["text"]

    scopes, scope_labels = detect_scopes(text)

    real_labels = test_data_with_scopes(data, i)

    print("Computed.")
    
    for true_value, pred_value in zip(real_labels, scope_labels):

      if true_value[1] == pred_value[1] and true_value[1]!= "O":
        true_positives+=1
            
      elif true_value[1] == pred_value[1]:
        true_negatives=1

      elif true_value[1]=="O" and pred_value[1]!="O":
        false_positives+=1
      
      else:
        false_negatives+=1
        
    i+=1

  true_negatives /= i
  true_positives /= i
  false_positives /= i
  false_negatives /= i

  # Measures

  denominator = true_positives + false_positives

  if denominator == 0:
    precision = 0
  
  else:
    precision = true_positives / denominator

  denominator = true_positives + false_negatives

  if denominator == 0:
    recall = 0

  else:
    recall = true_positives / denominator

  denominator = true_positives + true_negatives + false_positives + false_negatives

  if denominator == 0:
    f1_score = 0
    accuracy = 0

  else:
    f1_score = 2 * (precision * recall) / (precision + recall)
    accuracy = (true_positives + true_negatives) / denominator

  x = PrettyTable()
  x.field_names = ["Measure", "Score"]
  x.add_row(["Precision", f"{precision:.2f}"])
  x.add_row(["Recall", f"{recall:.2f}"])
  x.add_row(["F1-score", f"{f1_score:.2f}"])
  x.add_row(["Accuracy", f"{accuracy:.2f}"])

  print(x)

In [ ]:
import multiprocessing

def process_text(d, i):
    text = d["data"]["text"]
    scopes, scope_labels = detect_scopes(text)
    real_labels = test_data_with_scopes(data, i)
    
    return real_labels, scope_labels

def calculate_scores(data):
    true_negatives = 0
    true_positives = 0
    false_positives = 0
    false_negatives = 0

    pool = multiprocessing.Pool()

    results = []
    for i, d in enumerate(data):
        print("Processing text", i)
        results.append(pool.apply_async(process_text, args=(d, i)))

    pool.close()
    pool.join()

    for result in results:
        true_labels, pred_labels = result.get()

        for true_value, pred_value in zip(true_labels, pred_labels):
            if true_value[1] == pred_value[1] and true_value[1] != "O":
                true_positives += 1
            elif true_value[1] == pred_value[1]:
                true_negatives += 1
            elif true_value[1] == "O" and pred_value[1] != "O":
                false_positives += 1
            else:
                false_negatives += 1

    i = len(data)
    true_negatives /= i
    true_positives /= i
    false_positives /= i
    false_negatives /= i

    denominator = true_positives + false_positives
    if denominator == 0:
        precision = 0
    else:
        precision = true_positives / denominator

    denominator = true_positives + false_negatives
    if denominator == 0:
        recall = 0
    else:
        recall = true_positives / denominator

    denominator = true_positives + true_negatives + false_positives + false_negatives
    if denominator == 0:
        f1_score = 0
        accuracy = 0
    else:
        f1_score = 2 * (precision * recall) / (precision + recall)
        accuracy = (true_positives + true_negatives) / denominator

    x = PrettyTable()
    x.field_names = ["Measure", "Score"]
    x.add_row(["Precision", f"{precision:.2f}"])
    x.add_row(["Recall", f"{recall:.2f}"])
    x.add_row(["F1-score", f"{f1_score:.2f}"])
    x.add_row(["Accuracy", f"{accuracy:.2f}"])

    print(x)


## 🌸 Investigating the data

In [ ]:
for d in data:
  text = d["data"]["text"]
  predictions = d["predictions"][0]["result"]

  for value in predictions:
    if value["value"]["labels"][0] == "UNC":
      start_value = value['value']['start']
      end_value = value['value']['end']
      
      print(text[start_value:end_value])

## 🌸 Checking performance 

In [ ]:
#First, take one text from the sample and cut it to take only the important part

text = data[0]["data"]["text"]

#text = cut_text(text)


In [ ]:
#Then, does the cues detection work?

pred_labels, neg_cues, unc_cues = detect_cues(text)

In [ ]:
#Finally, are scopes correctly detected?

scopes, scope_labels = detect_scopes(text)


In [ ]:
#Just some visualization

scopes

[('sin',
  'antecedents - sin alergias mediamentosas conocidas - hipertension arterial en tratamiento farmaoclogico con tres farmacos .'),
 ('sin', 'ssf : viuda , sin hijos .'),
 ('no', ', no refiriendo una ideacion suicida clara'),
 ('sin',
  'epsontaneo y fluido , normoprosodico , sin alteraciones en el contenido de el pensamiento'),
 ('no', ', no evidenciando alteraciones endogenomorfas en el esatdo de animo'),
 ('no', 'no ideacion suicida estructurada en el momento actual .'),
 ('no',
  'no alteraciones en la sensopercepcion ni otras alteraciones dentro de la esfera psicotica .'),
 ('no', 'no auto ni heteroagresiva .')]

In [ ]:
scope_labels

[(nº, 'O'),
 (historia, 'O'),
 (clinica, 'O'),
 (:, 'O'),
 (**, 'O'),
 (***, 'O'),
 (***, 'O'),
 (nºepisodi, 'O'),
 (:, 'O'),
 (********, 'O'),
 (sexe, 'O'),
 (:, 'O'),
 (dona, 'O'),
 (data, 'O'),
 (de, 'O'),
 (naixement, 'O'),
 (:, 'O'),
 (06.06.1938, 'O'),
 (edat, 'O'),
 (:, 'O'),
 (79, 'O'),
 (anys, 'O'),
 (procedencia, 'O'),
 (domicil/res, 'O'),
 (.soc, 'O'),
 (servei, 'O'),
 (psiquiatria, 'O'),
 (data, 'O'),
 (d'ingres, 'O'),
 (10.05.2018, 'O'),
 (data, 'O'),
 (d'alta, 'O'),
 (10.05.2018, 'O'),
 (16:46:41, 'O'),
 (ates, 'O'),
 (per, 'O'),
 (****************, 'O'),
 (,, 'O'),
 (************, 'O'),
 (;, 'O'),
 (*************, 'O'),
 (,, 'O'),
 (assumpta, 'O'),
 (informe, 'O'),
 (d'alta, 'O'),
 (d'hospitalitzacio, 'O'),
 (motiu, 'O'),
 (d'ingres, 'O'),
 (paciente, 'O'),
 (de, 'O'),
 (79, 'O'),
 (años, 'O'),
 (que, 'O'),
 (acude, 'O'),
 (derivada, 'O'),
 (a, 'O'),
 (urgencias, 'O'),
 (de, 'O'),
 (psiquiatria, 'O'),
 (tras, 'O'),
 (sim, 'O'),
 (., 'O'),
 (antecedents, 'NSCO'),
 (-, 'NS

Here, we can see that the algorithm is able to capture all the negation and uncertainty cues in text 0:

In [ ]:
real_labels = test_data(data, 0)

Completed (0)
Completed (1)


In [ ]:
evaluating_model(real_labels, pred_labels, "NEG")

True Negatives: 600
True Positives: 8
False Positives: 0
False Negatives: 0
+-----------+-------+
|  Measure  | Score |
+-----------+-------+
| Precision |  1.00 |
|   Recall  |  1.00 |
|  F1-score |  1.00 |
|  Accuracy |  1.00 |
+-----------+-------+


The algorithm final performance in text 0. It will be different for all texts, but similar. At the end we can know the final measures by computing the mean for all the texts.

In [ ]:
real_labels = test_data_with_scopes(data, 0)

In [ ]:
final_evaluation(real_labels,scope_labels)

True Negatives: 1
True Positives: 54
False Positives: 5
False Negatives: 28
+-----------+-------+
|  Measure  | Score |
+-----------+-------+
| Precision |  0.92 |
|   Recall  |  0.66 |
|  F1-score |  0.77 |
|  Accuracy |  0.62 |
+-----------+-------+


In [ ]:
calculate_scores(data) #Although we are using multiprocessing, this function takes ages to run because it is computing for all texts (318) all the labels

Processing text 0
Processing text 1
Processing text 2
Processing text 3
Processing text 4
Processing text 5
Processing text 6
Processing text 7
Processing text 8
Processing text 9
Processing text 10
Processing text 11
Processing text 12
Processing text 13
Processing text 14
Processing text 15
Processing text 16
Processing text 17
Processing text 18
Processing text 19
Processing text 20
Processing text 21
Processing text 22
Processing text 23
Processing text 24
Processing text 25
Processing text 26
Processing text 27
Processing text 28
Processing text 29
Processing text 30
Processing text 31
Processing text 32
Processing text 33
Processing text 34
Processing text 35
Processing text 36
Processing text 37
Processing text 38
Processing text 39
Processing text 40
Processing text 41
Processing text 42
Processing text 43
Processing text 44
Processing text 45
Processing text 46
Processing text 47
Processing text 48
Processing text 49
Processing text 50
Processing text 51
Processing text 52
Pro

/usr/local/lib/python3.10/dist-packages/spacy/language.py:999: UserWarning: Due to multiword token expansion or an alignment issue, the original text has been replaced by space-separated expanded tokens.
  doc = self._ensure_doc(text)
/usr/local/lib/python3.10/dist-packages/spacy/language.py:999: UserWarning: Can't set named entities because of multi-word token expansion or because the character offsets don't map to valid tokens produced by the Stanza tokenizer:
Words: ['nº', 'historia', 'clinica', ':', '**', '***', '***', 'nºepisodi', ':', '********', 'sexe', ':', 'dona', 'data', 'de', 'naixement', ':', '15.07.1981', 'edat', ':', '36', 'anys', 'procedencia', 'domicil/res', '.soc', 'servei', 'otorrinolaringologia', 'data', "d'ingres", '16.05.2018', 'data', "d'alta", '18.05.2018', '10:23:26', 'ates', 'per', '*************', ',', '*****', 'informe', "d'alta", "d'hospitalitzacio", 'motiu', "d'ingres", 'sangrado', 'faringe', 'post', 'amigdalectomia', '.', 'antecedents', 'no', 'amc', 'ap', 

Completed (0)
Completed (1)
Completed (0)
Completed (1)


/usr/local/lib/python3.10/dist-packages/spacy/language.py:999: UserWarning: Can't set named entities because of multi-word token expansion or because the character offsets don't map to valid tokens produced by the Stanza tokenizer:
Words: ['nº', 'historia', 'clinica', ':', '**', '***', '***', 'nºepisodi', ':', '********', 'sexe', ':', 'dona', 'data', 'de', 'naixement', ':', '18.02.1991', 'edat', ':', '27', 'anys', 'procedencia', 'domicil/res', '.soc', 'servei', 'obstetricia', 'data', "d'ingres", '25.05.2018', 'data', "d'alta", '27.05.2018', '10:03:59', 'ates', 'per', '**************', ',', '**************', ';', '******************', ',', '*****', ';', '******', ';', '******************', ',', '*****', 'informe', "d'alta", "d'hospitalitzacio", 'motiu', "d'ingres", 'trabajo', 'de', 'parto', 'antecedents', 'no', 'alergias', 'medicamentosas', 'conocidas', '.', 'iq', ':', 'dedo', 'mano', '.', 'no', 'antecedentes', 'patologicos', '.', 'fumadora', 'de', '8-9', 'cig/dia', '.', 'medicacio', 'h

Completed (0)


...

/usr/local/lib/python3.10/dist-packages/spacy/language.py:999: UserWarning: Can't set named entities because of multi-word token expansion or because the character offsets don't map to valid tokens produced by the Stanza tokenizer:
Words: ['nº', 'historia', 'clinica', ':', '**', '***', '***', 'nºepisodi', ':', '********', 'sexe', ':', 'dona', 'data', 'de', 'naixement', ':', '14.11.1976', 'edat', ':', '42', 'anys', 'procedencia', 'domicil/res', '.soc', 'servei', 'obstetricia', 'data', "d'ingres", '08.09.2019', 'data', "d'alta", '11.09.2019', '11:17:28', 'ates', 'per', '***************', ',', '*****', ';', '*****************', ',', '*****', 'informe', "d'alta", "d'hospitalitzacio", 'motiu', "d'ingres", 'trabajo', 'de', 'parto', 'antecedents', 'no', 'alergias', 'medicamentosas', 'conocidas', 'antecedentes', 'patologicos', ':', 'hipotiroidismo', 'en', 'tratramiento', 'sustitutivo', 'intervenciones', 'quirurgicas', ':', 'amigdalectomia', ',', 'mamoplastia', 'de', 'reduccion', 'niega', 'habi

Completed (0)
Completed (0)
Completed (1)
Completed (1)


/usr/local/lib/python3.10/dist-packages/spacy/language.py:999: UserWarning: Can't set named entities because of multi-word token expansion or because the character offsets don't map to valid tokens produced by the Stanza tokenizer:
Words: ['nº', 'historia', 'clinica', ':', '**', '***', '***', 'nºepisodi', ':', '********', 'sexe', ':', 'home', 'data', 'de', 'naixement', ':', '26.08.2018', 'edat', ':', '12', 'mesos', 'procedencia', 'aten.primaria', 'servei', 'medicina', 'pediatrica', 'data', "d'ingres", '11.09.2019', 'data', "d'alta", '17.09.2019', '14:55:14', 'ates', 'per', '************', ',', '********', ';', '****************', ',', '*******', ';', '***********', ',', '******', 'informe', "d'alta", "d'hospitalitzacio", 'motiu', "d'ingres", 'paciente', 'de', '12', 'meses', 'que', 'ingresa', 'el', 'dia', '11/09', 'por', 'artritis', 'septica', 'de', 'cadera', 'derecha', '.', 'antecedents', 'no', 'alergias', 'medicamentosas', 'conocidas', '.', 'vacunas', 'a', 'el', 'dia', '.', 'padres', 

Completed (0)
Completed (0)
Completed (1)
Completed (1)


/usr/local/lib/python3.10/dist-packages/spacy/language.py:999: UserWarning: Can't set named entities because of multi-word token expansion or because the character offsets don't map to valid tokens produced by the Stanza tokenizer:
Words: ['nº', 'historia', 'clinica', ':', '**', '***', '***', 'nºepisodi', ':', '********', 'sexe', ':', 'dona', 'data', 'de', 'naixement', ':', '26.09.2018', 'edat', ':', '11', 'mesos', 'procedencia', 'domicil/res', '.soc', 'servei', 'medicina', 'pediatrica', 'data', "d'ingres", '12.09.2019', 'data', "d'alta", '14.09.2019', '15:16:00', 'ates', 'per', '****************', ',', '****', ';', '************', ',', '********', ';', '****************', ',', '******', ';', '****************', ',', '*******', ';', '****************', ',', '******', '.', 'informe', "d'alta", "d'hospitalitzacio", 'motiu', "d'ingres", 'paciente', 'de', '11', 'meses', 'que', 'ingresa', 'el', 'dia', '13/09', 'por', 'convulsion', 'febril', 'en', 'contexto', 'de', 'sospecha', 'de', 'bacteri